# 몬테카를로 컨트롤 구현
업그레이드된 그리디 월드에서 몬테카를로 컨트롤을 구현해보자.
이를 위해 간략화된 정책 이터레이션(평가와 개선 단계를 끝까지 진행하지 않고 얕게 진행하는 것)을 사용할 것이다. 즉, 굳이 수렴할 때까지 평가 단계를 진행하지 않고 일단 가치 테이블에 저장해둔 값들이 조금이라도 바뀌면 그에 대해 개선 단계를 진행해보자.

**수렴할 때까지 반복**
* 한 에피소드의 경험을 쌓고
* 경험된 데이터로 $ q(s,a) $ 테이블의 값을 업데이트하고(정책 평가)
* 업데이트된 $ q(s,a) $ 테이블을 이용하여 $ \epsilon -greedy $ 정책을 만들고(정책 개선)

위 과정을 반복한다.

## 라이브러리 import

In [2]:
import random
import numpy as np # q(s,a)를 numpy array 형태로 관리하기 위함

## GridWorld 클래스

In [9]:
class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0
    
    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a == 0:
            self.move_left()
        elif a == 1:
            self.move_up()
        elif a == 2:
            self.move_right()
        elif a == 3:
            self.move_down()
        
        reward = -1 # 보상은 -1로 고정
        done = self.is_done()
        return (self.x, self.y), reward, done
    
    def move_right(self):
        if self.y == 1 and self.x in [0, 1, 2]:
            pass
        elif self.y == 3 and self.x in [2, 3, 4]:
            pass
        elif self.y == 6:
            pass
        else:
            self.y += 1
        
    def move_left(self):
        if self.y == 0:
            pass
        elif self.y == 3 and self.x in [0, 1, 2]:
            pass
        elif self.y == 5 and self.x in [2, 3, 4]:
            pass
        else:
            self.y -= 1
    
    def move_up(self):
        if self.x == 0:
            pass
        elif self.x == 3 and self.y == 2:
            pass
        else:
            self.x -= 1
            
    def move_down(self):
        if self.x == 4:
            pass
        elif self.x == 1 and self.y == 4:
            pass
        else:
            self.x += 1
    
    def is_done(self):
        if self.x == 4 and self.y == 6: # 목표 지점인 (4, 6)에 도달
            return True
        else:
            return False
    
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

## QAgent 클래스

에이전트에 해당하는 객체를 정의해보자.
에이전트 객체는 내부에 $q(s,a)$의 값을 저장하기 위한 테이블을 갖고 있고, 이 테이블은 실제로 에이전트가 액션을 선택할 때 사용된다. 

* `select_action` 함수를 통해 상태 $s$를 인풋으로 받아 $s$에서 알맞은 액션을 입실론 그리디 방식을 통해 선택한다.
* `update_table` 함수는 실제로 테이블의 값을 업데이트 해주는 함수로, 하나의 에피소드에 해당하는 데이터를 받아서 MC 방법으로 테이블의 값을 업데이트 한다.
* `anneal_eps` 함수는 epsilon의 값을 조금씩 줄여주는 함수이다.
* `show_table` 함수는 학습이 끝난 후에 상태별로 $q(s,a)$의 값이 가장 큰 액션을 뽑아서 보여주는 함수이다.

In [13]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) # q밸류를 저장하는 변수. 모두 0으로 초기화
        self.eps = 0.9
        self.alpha = 0.01
    
    def select_action(self, s):
        # eps-greedy로 액션을 선택
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0, 3)
        else:
            action_val = self.q_table[x, y,:]
            action = np.argmax(action_val)
        return action
    
    def update_table(self, history):
        # 한 에피소드에 해당하는 history를 입력으로 받아 q 테이블의 값을 업데이트
        cum_reward = 0
        for transition in history[::-1]:
            s, a, r, s_prime = transition
            x, y = s
            # 몬테카를로 방식을 사용하여 업데이트
            self.q_table[x, y, a] = self.q_table[x, y, a] + self.alpha * (cum_reward - self.q_table[x, y, a])
            cum_reward = cum_reward + r
            
    def anneal_eps(self):
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)
    
    def show_table(self):
        # 학습이 각 위치에서 어느 액션의 q 값이 가장 높았는지 보여줌
        q_list = self.q_table.tolist()
        data = np.zeros((5, 7))
        for row_idx in range(len(q_list)):
            row = q_list[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

## Main 함수

에이전트와 환경을 만들고, 하나의 에피소드가 끝날 때까지 `history`라는 변수에 상태 전이 과정을 모두 저장해두었다가, 에피소드가 끝나는 순간 해당 변수를 이용해서 에이전트 내부의 q 테이블을 업데이트한다. 그리고 epsilon의 값을 조금씩 줄여준다.
1천 번의 에피소드 동안 학습을 하고 나면 최종 결과를 출력해준다.

In [5]:
def main():
    env = GridWorld()
    agent = QAgent()
    
    for n_epi in range(1000): # 총 1000 에피소드 동안 학습
        done = False
        history = []
        
        s = env.reset()
        while not done: # 한 Q에피소드가 끝날 때까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            history.append((s, a, r, s_prime))
            s = s_prime
        agent.update_table(history)
        agent.anneal_eps()
        
    agent.show_table() # 학습이 끝난 결과를 출력

In [14]:
main()

[[3. 0. 0. 2. 2. 3. 3.]
 [2. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 1. 0. 3. 0.]
 [0. 3. 1. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 2. 0.]]
